<a href="https://colab.research.google.com/github/Ads369/Ads_2s/blob/main/ipynb/Lesson%2026/26_2_%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5_%D0%B8%D1%81%D0%BF%D0%BE%D0%BB%D1%8C%D0%B7%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_REST_API_%D0%B2_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Навигация по уроку**

1. [Веб-архитектура сервиса](https://colab.research.google.com/drive/10wtDodlf4SaVcYk6VoXDWk650IDcNPaa)
2. Практическое использование REST API в Python
3. [Введение в FastAPI](https://colab.research.google.com/drive/1_AzAVys4xub3yyw763NDwfeJ3WecGgkb)
5. [Домашняя работа](https://colab.research.google.com/drive/1SlJW51-OaUUDPuk-j9GYNHHwFkHBztU_)

## Реализация REST API в Python

Наиболее популярными библиотеками для работы с **HTTP** в **Python** являются **urllib**, **urllib2**, **httplib** и **requests**. Для доступа к низкоуровневым функциям протокола используют **socket**.

Самой удобной библиотекой принято считать **requests**, потому ее все чаще включают в свой инструментарий не только программисты, но и специалисты в области DataScience.





### Библиотека **REQUESTS**

Рассмотрим реализацию основных методов **REST API** с использованием модуля **requests**.

Попробуем получить веб-страницу с помощью **GET**-запроса.

**GET** является одним из самых популярных **HTTP**-методов. Метод **GET** указывает на то, что происходит попытка извлечь данные из определенного ресурса. Для того, чтобы выполнить запрос **GET**, используется метод `requests.get()`.

Для проверки работы команды мы будет использовать [Root REST API](https://docs.github.com/ru/rest?apiVersion=2022-11-28#root-endpoint) на GitHub. Для указанного ниже URL вызывается метод `get()`:

In [ ]:
import requests
response = requests.get('https://api.github.com')

В переменную `response` мы поместили значение объекта **Response** и теперь можем рассмотреть более детально данные, которые были получены в результате запроса **GET**.



**HTTP коды состояний**

Мы уже познакомились с кодами состояний в первой части урока и знаем, что если запрос был успешно выполнен, то получим код состояния `200`. Проверим данное утверждение. Для этого необходимо обратиться к атрибуту `status_code`, полученного объекта:

In [ ]:
response.status_code

200


```

`status_code` вернул значение `200`. Это значит, что запрос был выполнен успешно!

Код состояния часто используется, чтобы задать разную логику программы в зависимости от результата запроса:

In [ ]:
if response.status_code == 200:
    print('Данные получены!')
elif response.status_code == 404:
    print('Упс! Попробуйте снова!')

Данные получены!


Библиотека `requests` предоставляет возможность использовать и более простую конструкцию. Если использовать полученный объект `Response` в условных конструкциях, то при получении кода состояния в промежутке от  `200` до `400`, будет выведено значение `True`. В противном случае отобразится значение `False`.

Последний пример можно упростить при помощи использования оператора `if`:

In [ ]:
if response:
    print('Данные получены!')
else:
    print('Упс! Попробуйте снова!')

Данные получены!


Стоит иметь в виду, что данный способ не проверяет, имеет ли статусный код точное значение `200`. Причина заключается в том, что другие коды в промежутке от `200` до `400`, например, `204 NO CONTENT` и `304 NOT MODIFIED`, также считаются успешными в случае, если они могут предоставить действительный ответ.

К примеру, код состояния `204` говорит о том, что ответ успешно получен, однако в полученном объекте нет содержимого. Можно сказать, что для оптимально эффективного использования способа необходимо убедиться, что начальный запрос был успешно выполнен. Требуется изучить код состояния и в случае необходимости произвести необходимые поправки, которые будут зависеть от значения полученного кода.

Если вы не хотите использовать оператора `if` для проверки кода состояния, то можете генерировать исключения при неудачных запросах с помощью метода `raise_for_status()`:

In [ ]:
import requests
# импорт ошибки метода HTTP
from requests.exceptions import HTTPError

# В цикле обращаемся к двум URI: реальному и вымышленному
for url in ['https://api.github.com', 'https://api.github.com/invalid']:
    print(f'Запрос по адресу: {url}')
    # Блок обработки исключений (ошибок)
    try:
        # выполняем GET запрос
        response = requests.get(url)

        # если ответ успешен, исключения задействованы не будут
        response.raise_for_status()
    except HTTPError as http_err:
        # Если ошибка связана с HTTP запросом, то выполнится этот блок HTTPError
        print(f'HTTP ошибка: {http_err}')
    except Exception as err:
        # Если ошибка не связана с HTTP запросом, то выполнится этот блок Exception
        print(f'Другая ошибка (не HTTP): {err}')
    else:
        # При успешном выполнении после блока try выполнится данный блок else
        print('Это успех!')

Запрос по адресу: https://api.github.com
Это успех!
Запрос по адресу: https://api.github.com/invalid
HTTP ошибка: 404 Client Error: Not Found for url: https://api.github.com/invalid


Таким образом при вызова метода `raise_for_status()` проверяется код состояния, и для некоторых кодов вызывается исключение `HTTPError`. Если код состояния относится к успешным, то программа продолжает выполнение.

**Содержимое страницы**

Делая GET запрос, мы ожидаем получить ценную для нас информацию. Эта информация, как правило, находится в теле сообщения и называется **пейлоад (payload)**. Используя атрибуты и методы объекта **Response**, можно извлечь информацию из пайлоад в различных форматах.

Для того, чтобы получить содержимое запроса в байтах, необходимо обратиться к атрибуту `content`:

In [ ]:
response.content

b'{"message":"Not Found","documentation_url":"https://docs.github.com/rest"}'

Использование `content` обеспечивает доступ к чистым байтам ответного пейлоада, то есть к любым данным в теле запроса. Однако, зачастую требуется получить информацию в виде строки в кодировке `UTF-8`, что можно сделать, обратившись к атрибуту `text`:

In [ ]:
response.text

'{"message":"Not Found","documentation_url":"https://docs.github.com/rest"}'

Декодирование байтов в строку требует наличия определенной кодировки. По умолчанию **requests** попытается узнать текущую кодировку, ориентируясь по заголовкам HTTP. Указать необходимую кодировку можно при помощи добавления `encoding` перед `text`:

In [ ]:
response.encoding = 'utf-8' # Сообщаем, что нам нужна кодировка utf-8
response.text

Если присмотреться к ответу, то можно заметить, что его содержимое является сериализированным (преобразованным в текст) JSON контентом. Воспользовавшись библиотекой `json`, можно взять полученные из `text` строки `str` и провести с ними обратную сериализацию (преобразовать в JSON) при помощи использования `json.loads()`:

In [ ]:
import json
json.loads(response.text)

{'message': 'Not Found', 'documentation_url': 'https://docs.github.com/rest'}

Есть и более простой способ с использованием  метода `json()`:

In [ ]:
response.json()

{'message': 'Not Found', 'documentation_url': 'https://docs.github.com/rest'}

Тип полученного значения методом `json()`, является словарем. А значит, доступ к его содержимому можно получить по ключу.

**HTTP-заголовки**

**HTTP-заголовки** ответов на запрос могут содержать полезную информацию. Это может быть тип содержимого ответного пейлоада, либо ограничение по времени для кеширования ответа и многое другое.

Для просмотра HTTP заголовков необходимо обратиться к атрибуту `headers`:



In [ ]:
response.headers

{'Server': 'GitHub.com', 'Date': 'Mon, 29 Apr 2024 17:45:11 GMT', 'Content-Type': 'application/json; charset=utf-8', 'X-GitHub-Media-Type': 'github.v3; format=json', 'x-github-api-version-selected': '2022-11-28', 'Access-Control-Expose-Headers': 'ETag, Link, Location, Retry-After, X-GitHub-OTP, X-RateLimit-Limit, X-RateLimit-Remaining, X-RateLimit-Used, X-RateLimit-Resource, X-RateLimit-Reset, X-OAuth-Scopes, X-Accepted-OAuth-Scopes, X-Poll-Interval, X-GitHub-Media-Type, X-GitHub-SSO, X-GitHub-Request-Id, Deprecation, Sunset', 'Access-Control-Allow-Origin': '*', 'Strict-Transport-Security': 'max-age=31536000; includeSubdomains; preload', 'X-Frame-Options': 'deny', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '0', 'Referrer-Policy': 'origin-when-cross-origin, strict-origin-when-cross-origin', 'Content-Security-Policy': "default-src 'none'", 'Vary': 'Accept-Encoding, Accept, X-Requested-With', 'Content-Encoding': 'gzip', 'X-RateLimit-Limit': '60', 'X-RateLimit-Remaining': '52

Ответ в формате "словаря" (почти), в таком виде сложно читается. Для упрощения восприятия можно сериализовать данные с помощью `json.dumps()` с параметром `indent` (число отступов в пробелах при форматировании):

In [ ]:
print(json.dumps(dict(response.headers), indent=4))

{
    "Server": "GitHub.com",
    "Date": "Mon, 29 Apr 2024 17:45:11 GMT",
    "Content-Type": "application/json; charset=utf-8",
    "X-GitHub-Media-Type": "github.v3; format=json",
    "x-github-api-version-selected": "2022-11-28",
    "Access-Control-Expose-Headers": "ETag, Link, Location, Retry-After, X-GitHub-OTP, X-RateLimit-Limit, X-RateLimit-Remaining, X-RateLimit-Used, X-RateLimit-Resource, X-RateLimit-Reset, X-OAuth-Scopes, X-Accepted-OAuth-Scopes, X-Poll-Interval, X-GitHub-Media-Type, X-GitHub-SSO, X-GitHub-Request-Id, Deprecation, Sunset",
    "Access-Control-Allow-Origin": "*",
    "Strict-Transport-Security": "max-age=31536000; includeSubdomains; preload",
    "X-Frame-Options": "deny",
    "X-Content-Type-Options": "nosniff",
    "X-XSS-Protection": "0",
    "Referrer-Policy": "origin-when-cross-origin, strict-origin-when-cross-origin",
    "Content-Security-Policy": "default-src 'none'",
    "Vary": "Accept-Encoding, Accept, X-Requested-With",
    "Content-Encoding": "g

Прежде чем значение `response.headers` сериализовать, мы его привели к типу данных `dict`, так как он имеет тип отличный от привычного нам словаря:

In [ ]:
type(response.headers)

requests.structures.CaseInsensitiveDict

Он похож на словарь, и многие методы к нему применимы, как со словарем, но когда доходит дело до сериализации (то есть преобразование к JSON подобному тексту), должен быть на входе именно словарь (а не словарь подобный объект).  

Так как `headers` возвращает объект подобный словарю, то мы можем получить доступ к значению заголовка HTTP по ключу. Например, для просмотра типа содержимого ответного пейлоада, требуется использовать `Content-Type`.

In [ ]:
response.headers['Content-Type']

'application/json; charset=utf-8'

Специфика HTTP предполагает, что заголовки не чувствительны к регистру. Это значит, что при получении доступа к заголовкам можно не беспокоится о том, использованы строчные или прописные буквы.

In [ ]:
response.headers['content-type']

'application/json; charset=utf-8'

При использовании ключей `content-type` и `Content-Type` результат будет получен один и тот же. Таким свойством обычный словарь не обладает!

**Параметры запроса**

Наиболее простым способом выполнить запрос **GET** с параметрами является передача значений через параметры строки запроса в URI. При использовании метода `get()`, данные передаются в `params`:

In [ ]:
import requests

# Поиск репозитариев c упоминание ключевого слова requests и языка Python на GitHub
response = requests.get(
    'https://api.github.com/search/repositories', # URI
    params={'q': 'requests+language:python'},     # Параметры для запроса
)

# Данные в формате JSON
json_response = response.json()

# Первый найденный репозитарий
repository = json_response['items'][0]


print(f'Имя репозитария: {repository["name"]}')
print(f'Описание репозитария: {repository["description"]}')

Имя репозитария: secrules-language-evaluation
Описание репозитария: Set of Python scripts to perform SecRules language evaluation on a given http request.


Параметры можно передавать в форме словаря:
```python
params={'q': 'requests+language:python'}
```

в форме списка кортежей:
```python
params=[('q', 'requests+language:python')]
```

или передать значение в байтах:
```python
params=b'q=requests+language:python'
```

**Настройка HTTP-заголовка запроса (headers)**

Для изменения HTTP-заголовка требуется передать словарь данного HTTP-заголовка в `get()` при помощи использования параметра `headers`.

Например, можно взять предыдущий пример, и указать ГитХабу (GitHub), что необходимо подсветить в ответе все места с найденным текстом как в запросе (`requests+language:python`).

Для этого в заголовке `Accept` передается тип `text-match`, понятный ГитХабу.


In [ ]:
import requests

response = requests.get(
    'https://api.github.com/search/repositories',
    params={'q': 'requests+language:python'},
    headers={'Accept': 'application/vnd.github.v3.text-match+json'},
)

# просмотр нового массива `text-matches` с предоставленными данными
# о поиске в пределах результатов
json_response = response.json()
repository = json_response['items'][0]
repository["text_matches"]

[{'object_url': 'https://api.github.com/repositories/33210074',
  'object_type': 'Repository',
  'property': 'description',
  'fragment': 'Set of Python scripts to perform SecRules language evaluation on a given http request.',
  'matches': [{'text': 'Python', 'indices': [7, 13]},
   {'text': 'language', 'indices': [42, 50]},
   {'text': 'request', 'indices': [78, 85]}]}]

Заголовок `Accept` сообщает серверу о типах контента, который можно использовать в рассматриваемом приложении. Здесь подразумевается, что все совпадения будут подсвечены, для чего в заголовке используется значение `application/vnd.github.v3.text-match+json`. Это уникальный заголовок `Accept` для GitHub. В данном случае содержимое представлено в специальном JSON формате.

**HTTP-методы в requests**

Помимо **GET**, в библиотеке реализованы и другие HTTP-методы, такие как **POST**, **PUT**, **DELETE**, **HEAD**, **PATCH** и **OPTIONS**. Для каждого из этих методов существует своя структура запросов, которая очень похожа на метод `get()`.

In [ ]:
requests.post('https://httpbin.org/post', data={'key':'value'})
requests.put('https://httpbin.org/put', data={'key':'value'})
requests.delete('https://httpbin.org/delete')
requests.head('https://httpbin.org/get')
requests.patch('https://httpbin.org/patch', data={'key':'value'})
requests.options('https://httpbin.org/get')

<Response [200]>

Несмотря на отличия в HTTP-методах, схема работы с ответами будет аналогична, как мы работали с GET-запросом.

Каждая функция создает запрос к **httpbin.org** сервису, используя при этом ответный **HTTP-метод**. Это удобный сервис для тестирования REST API.
Это чрезвычайно полезный сервис, созданный человеком, который внедрил использование `requests` – Кеннетом Рейтцом. Данный сервис предназначен для тестовых запросов. Здесь можно составить пробный запрос и получить ответ с требуемой информацией.

In [ ]:
response = requests.head('https://httpbin.org/get')
print(f'Content-Type: {response.headers["Content-Type"]}')


response = requests.delete('https://httpbin.org/delete')
response.json()

Content-Type: application/json


{'args': {},
 'data': '',
 'files': {},
 'form': {},
 'headers': {'Accept': '*/*',
  'Accept-Encoding': 'gzip, deflate',
  'Content-Length': '0',
  'Host': 'httpbin.org',
  'User-Agent': 'python-requests/2.31.0',
  'X-Amzn-Trace-Id': 'Root=1-6630069d-35f5cba332ee5eab68de8244'},
 'json': None,
 'origin': '35.227.136.8',
 'url': 'https://httpbin.org/delete'}

При использовании каждого из данных методов в объекте **Response** могут быть возвращены заголовки, тело запроса, коды состояния и многие другие аспекты.

Методы **HEAD**, **PATCH** и **OPTIONS** не используются в **REST API**, поэтому мы не будем на них останавливаться.





**Передача параметров через тело запроса**

В соответствии со спецификацией HTTP запросы **POST**, **PUT** и **PATCH** передают информацию через тело запроса, а не через параметры строки запроса. Используя библиотеку `requests`, можно передать данные в параметр `data`.

В свою очередь `data` использует словарь, список кортежей, байтов или объект файла. Это особенно важно, так как может возникнуть необходимость адаптации отправляемых с запросом данных в соответствии с определенными параметрами сервера.

К примеру, если тип содержимого запроса `application/x-www-form-urlencoded`, следует отправлять данные формы в виде словаря.



In [ ]:
request = requests.post('https://httpbin.org/post', data={'key':'value'})
request.json()

{'args': {},
 'data': '',
 'files': {},
 'form': {'key': 'value'},
 'headers': {'Accept': '*/*',
  'Accept-Encoding': 'gzip, deflate',
  'Content-Length': '9',
  'Content-Type': 'application/x-www-form-urlencoded',
  'Host': 'httpbin.org',
  'User-Agent': 'python-requests/2.31.0',
  'X-Amzn-Trace-Id': 'Root=1-663008ce-1c24eb3e244d91b6251e1d8c'},
 'json': None,
 'origin': '35.227.136.8',
 'url': 'https://httpbin.org/post'}

Ту же самую информацию также можно отправить в виде списка кортежей:

In [ ]:
request = requests.post('https://httpbin.org/post', data=[('key', 'value')])
request.json()

{'args': {},
 'data': '',
 'files': {},
 'form': {'key': 'value'},
 'headers': {'Accept': '*/*',
  'Accept-Encoding': 'gzip, deflate',
  'Content-Length': '9',
  'Content-Type': 'application/x-www-form-urlencoded',
  'Host': 'httpbin.org',
  'User-Agent': 'python-requests/2.31.0',
  'X-Amzn-Trace-Id': 'Root=1-6630097c-19af246a1c6896140ee473e7'},
 'json': None,
 'origin': '35.227.136.8',
 'url': 'https://httpbin.org/post'}

В том случае, если требуется отравить данные **JSON**, можно использовать параметр `json`. При передачи данных **JSON** через `json`, `requests` произведет сериализацию данных и добавит правильный `Content-Type` заголовок.

In [ ]:
response = requests.post('https://httpbin.org/post', json={'key':'value'})
json_response = response.json()
print(f'Отправленные данные: {json_response["data"]}')
print(f'Заголовок: {json_response["headers"]["Content-Type"]}')

Отправленные данные: {"key": "value"}
Заголовок: application/json


Мы видим, что сервер получил данные и HTTP заголовки, отправленные вместе с запросом. `requests` также предоставляет информацию в форме `PreparedRequest` (подготовленных к отправке данных).

**PreparedRequest (подготовленных данных)**

При составлении запроса стоит иметь в виду, что перед его фактической отправкой на целевой сервер библиотека `requests` выполняет определенную подготовку. Подготовка запроса включает в себя такие вещи, как проверка заголовков и сериализация содержимого JSON.

Если обратиться к атрибуту `request`, то можно просмотреть объект **PreparedRequest** (подготовленных данных):

In [ ]:
response = requests.post('https://httpbin.org/post', json={'key':'value'})

print('Подготовленные к отправке данные: ')
print('Content-Type: ', response.request.headers['Content-Type'])
print('URI: ', response.request.url)
print('Тело запроса: ', response.request.body)

Подготовленные к отправке данные: 
Content-Type:  application/json
URI:  https://httpbin.org/post
Тело запроса:  b'{"key": "value"}'


Проверка **PreparedRequest** открывает доступ ко всей информации о выполняемом запросе. Это может быть пейлоад, URI, заголовки, аутентификация и многое другое.

У всех описанных ранее типов запросов была одна общая черта – они представляли собой неаутентифицированные запросы к публичным API. Однако, подобающее большинство служб, с которыми может столкнуться пользователь, запрашивают аутентификацию.

#### Аутентификация HTTP AUTH

Аутентификация помогает сервису понять, кто вы. Как правило, вы предоставляете свои учетные данные на сервер, передавая данные через заголовок `Authorization` или пользовательский заголовок, определенной службы.

Одним из примеров API, который требует аутентификации, является [Authenticated User API](https://docs.github.com/ru/rest/users?apiVersion=2022-11-28#get-the-authenticated-user) на GitHub. Это раздел веб-сервиса, который предоставляет информацию о профиле аутентифицированного пользователя. Чтобы отправить запрос API-интерфейсу аутентифицированного пользователя, вы можете передать свое имя пользователя и пароль на GitHub через кортеж в  методе `get()`.

In [ ]:
from getpass import getpass # ввод пароля в колабе
requests.get('https://api.github.com/user', auth=('username', getpass()))

··········


<Response [401]>

Запрос будет выполнен успешно, если учетные данные, которые вы передали в кортеже `auth`, соответствуют реальному пользователю ГитХаба. Если выполнить запрос без учетных данных или с неверными данными, то получим код состояния **401 Unauthorized**.

Теперь, когда вы знаете о REST API практически все, пора [приступить](https://colab.research.google.com/drive/1_AzAVys4xub3yyw763NDwfeJ3WecGgkb) к самой главной части урока - создание своего собственного веб-сервиса на базе фреймворка FastAPI.